In [1]:
import pandas as pd
import math
from sklearn.cross_validation import train_test_split
import numpy as np
from sklearn.linear_model import logistic
import matplotlib.pyplot  as plt
import statsmodels.api as sm 
import statsmodels.discrete.discrete_model as dm

a = logistic.LogisticRegression()

C:\Users\tips1660\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
ml_congs = list(range(111,112))
#individual congresses
from sklearn.linear_model import logistic
for cong in ml_congs:
    print("---------------------CONG: %d-------------------" %(cong))
    df = pd.read_csv(str(cong)+"mlready_topic16.csv", encoding='latin-1')   
    avg= df['same_vote_prop'].mean()
    print("--------------------AVG------------------")
    print("for cong: %d avg is :%.2f" %(cong, avg))
    df['same_vote'] = df['same_vote_prop'].apply(lambda x: 0 if x<avg+.1 else 1)
    cols = [a for a in df if 'vote' not in a and 'cosponsored' not in a and 'named' not in a]
    df_final_split = df[cols]
    df_y = df['same_vote']
    
    x_train, x_test, y_train, y_test = train_test_split(df_final_split, df_y)
    mod = sm.Logit(y_train, x_train)
    mod2 = dm.Logit(y_train, x_train)
    res2 = mod2.fit()
    print("Summary")
    print(res2.summary())
    print("P Values")
    print(res2.pvalues)
    print("marg effs") # gives the  change in probability of success
    print(res2.get_margeff().summary())
    print("ODDS RATIOS") #gives the change in odds of success
    odds = np.exp(res2.params)
    print(odds)
    print("predicted probabilities")
    print(odds/(1+odds))
    print("-----------------------------------------")

---------------------CONG: 111-------------------
--------------------AVG------------------
for cong: 111 avg is :0.64
Optimization terminated successfully.
         Current function value: 0.234524
         Iterations 8
Summary
                           Logit Regression Results                           
Dep. Variable:              same_vote   No. Observations:                61503
Model:                          Logit   Df Residuals:                    61496
Method:                           MLE   Df Model:                            6
Date:                Tue, 24 Apr 2018   Pseudo R-squ.:                  0.6524
Time:                        14:57:57   Log-Likelihood:                -14424.
converged:                       True   LL-Null:                       -41501.
                                        LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------

In [3]:
#stacked
def large_dataframe(cols_in_general,cols_to_fill, df_orig,cong):
    df_returned = pd.DataFrame()
    cols_to_take = []
    for col in cols_to_fill:
        col_sin_cong = col.replace('_'+str(cong),"")
        cols_to_take.append(col_sin_cong)
        
    df_returned = df_orig[cols_to_take]
    df_returned['same_vote_prop'] = df_orig['same_vote_prop']
    cols_to_fill.append("same_vote_prop")
    df_returned.columns = cols_to_fill
    return df_returned

topics = [a for a in list(range(1,24)) if a!=11 and a!=22]
columns_school = ['same_school_'+str(topic) for topic in topics]
columns_gender = ['same_gender_'+str(topic) for topic in topics]
columns_rep = ['diff_rep_'+str(topic) for topic in topics]
columns_dem = ['diff_dem_'+str(topic) for topic in topics]
columns_state = ['same_state_'+str(topic) for topic in topics]
columns_party = ['same_party_'+str(topic) for topic in topics]
columns_leader = ['both_leader_'+str(topic) for topic in topics]
columns_party.extend(columns_school)
columns_party.extend(columns_gender)
columns_party.extend(columns_rep)
columns_party.extend(columns_dem)
columns_party.extend(columns_state)
columns_party.extend(columns_leader)
df_list_train = []
df_list_out_of_sample = []

for topic in topics:
    df = pd.read_csv("111mlready_topic"+str(topic)+".csv", encoding='latin-1')
    returned_df = large_dataframe(columns_party, ['same_party_'+str(topic), 'same_school_'+str(topic), 'diff_rep_'+str(topic), 'diff_dem_'+str(topic),'same_gender_'+str(topic), 'same_state_'+str(topic), 'both_leader_'+str(topic)], df, topic)
    df_list_train.append(returned_df)

    

C:\Users\tips1660\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
other_dfs  = df_list_train[1:]
final_df = df_list_train[0].append(other_dfs)
final_df = final_df.fillna(0)
avg= final_df['same_vote_prop'].mean()
std = final_df['same_vote_prop'].std()
final_df['same_vote'] = final_df['same_vote_prop'].apply(lambda x: 0 if x<avg+std else 1)
cols = [a for a in final_df if 'vote' not in a and 'cosponsored' not in a and 'named' not in a]
df_final_split = final_df[cols]
df_y = final_df['same_vote']
mod2 = dm.Logit(df_y, df_final_split)
res2 = mod2.fit()
print("Summary")
print(res2.summary())
print("marg effs") # gives the  change in probability of success
print(res2.get_margeff(at = 'mean').summary())

print("ODDS RATIOS") #gives the change in odds of success
odds = np.exp(res2.params)
print(odds)



Optimization terminated successfully.
         Current function value: 0.355682
         Iterations 8
Summary
                           Logit Regression Results                           
Dep. Variable:              same_vote   No. Observations:              1499202
Model:                          Logit   Df Residuals:                  1499055
Method:                           MLE   Df Model:                          146
Date:                Tue, 24 Apr 2018   Pseudo R-squ.:                  0.2872
Time:                        15:17:14   Log-Likelihood:            -5.3324e+05
converged:                       True   LL-Null:                   -7.4811e+05
                                        LLR p-value:                     0.000
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
both_leader_1     -0.0166      0.175     -0.095      0.924      -0.360       0.327
both_lead